In [8]:
import os
import time
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains



In [9]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"

In [10]:
url = "https://www.bigbasket.com/pd/1204455/fortune-sunflower-refined-oil-sun-lite-3x910-g/?nc=l3category&t_pg=L3Categories&t_p=l3category&t_s=l3category&t_pos=3&t_ch=desktop"

In [11]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument(f'user_agent={user_agent}')
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-infobars")
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
wait = WebDriverWait(driver, 10)

In [12]:
# 1. Category, Sub-Category, and Product
div_category = driver.find_element(By.CSS_SELECTOR, "div._3moNK")
a_tags = div_category.find_elements(By.CSS_SELECTOR, "a._3WUR_._3bj9B.rippleEffect")[1:]
category, sub_category, product = [a_tag.text.replace(">", "").strip() for a_tag in a_tags]

# 2. SKU ID
sku_id = url.split("/")[-3]

# 3. Image
image_div = driver.find_element(By.CSS_SELECTOR, "div._2FbOx")
image = image_div.find_element(By.CSS_SELECTOR, "img._3oKVV").get_attribute("src")

# 4. Brand
brand_div = driver.find_element(By.CSS_SELECTOR, "div._2yfKw")
brand = brand_div.find_element(By.CSS_SELECTOR, "a._2zLWN._3bj9B.rippleEffect").text

# 5. SKU Name and SKU Size
sku_div = driver.find_element(By.CSS_SELECTOR, "div._2yfKw")
sku_string = sku_div.find_element(By.CSS_SELECTOR, "h1.GrE04").text
sku_name = sku_string.split(" - ")[0].strip()
sku_size = sku_string.split(",")[-1].strip()

# 6. MRP and Selling Price
price_div = driver.find_element(By.CSS_SELECTOR, "div#price")
try:
    mrp_element = price_div.find_element(By.CSS_SELECTOR, "td._2ifWF")
    mrp = mrp_element.text
except NoSuchElementException:
    mrp = "NA"
selling_price_element = price_div.find_element(By.CSS_SELECTOR, "td.IyLvo")
selling_price = selling_price_element.text

# 7. Out of Stock
try:
    driver.find_element(By.CSS_SELECTOR, "div._36qqs")
    in_stock = "No"
except NoSuchElementException:
    in_stock = "Yes"
out_of_stock = "Yes" if in_stock == "No" else "No"

# 8. Product Rating
try:
    product_rating_element = driver.find_element(By.CSS_SELECTOR, "div._2Ze34")
    product_rating = product_rating_element.text
except (NoSuchElementException, TimeoutException):
    product_rating = "NA"

# 9. Total Ratings and Reviews
try:
    review_div = driver.find_element(By.CSS_SELECTOR, "div._1AXTE")
    reviews_element = review_div.find_element(By.CSS_SELECTOR, "span[style='color: rgb(74, 74, 74); position: relative; top: 1px;']")
    total_reviews_ratings = reviews_element.text
except NoSuchElementException:
    total_reviews_ratings = "NA"

# Separate Total Ratings and Total Reviews
if total_reviews_ratings != "NA":
    total_ratings, total_reviews = total_reviews_ratings.split('&')
    total_ratings = total_ratings.strip()[:-8]
    total_reviews = total_reviews.strip()[:-8]  
else:
    total_ratings = total_reviews = "NA"


# 10. Reviews
try:
    reviews_link_element = driver.find_element(By.CSS_SELECTOR, "a._1xG1d")
    reviews_link = reviews_link_element.get_attribute('href')
    driver.get(reviews_link)

    actions = ActionChains(driver)
    old_scroll_height = 0

    while True:
        actions.send_keys(Keys.END).perform()
        sleep(5)
        new_scroll_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        if new_scroll_height == old_scroll_height:
            break
        else:
            old_scroll_height = new_scroll_height

    # Extract all reviews
    review_elements = driver.find_elements(By.CSS_SELECTOR, "div.zF-ya")
    reviews = [review_element.text for review_element in review_elements]
except NoSuchElementException:
    reviews = []


data = {
    "Category": category,
    "Sub-Category": sub_category,
    "Product": product,
    "SKU ID": sku_id,
    "Image": image,
    "Brand": brand,
    "SKU Name": sku_name,
    "SKU Size": sku_size,
    "MRP": mrp,
    "Selling Price": selling_price,
    "In Stock": in_stock,
    "Out of Stock": out_of_stock,
    "Product Rating": product_rating,
    "Total Ratings": total_ratings,
    "Total Reviews": total_reviews,
    "Reviews": reviews
}


In [13]:
data

{'Category': 'FOODGRAINS, OIL & MASALA',
 'Sub-Category': 'EDIBLE OILS & GHEE',
 'Product': 'SUNFLOWER, RICE BRAN OIL',
 'SKU ID': '1204455',
 'Image': 'https://www.bigbasket.com/media/uploads/p/l/1204455_2-fortune-sunflower-refined-oil-sun-lite.jpg',
 'Brand': 'Fortune',
 'SKU Name': 'Fortune Sunflower Refined Oil',
 'SKU Size': '3x910 g Multipack',
 'MRP': 'Rs 420',
 'Selling Price': 'Rs 372',
 'In Stock': 'Yes',
 'Out of Stock': 'No',
 'Product Rating': '4.1',
 'Total Ratings': '43247',
 'Total Reviews': '633',
 'Reviews': ['good product ?',
  'Regular User',
  'We enjoy this oil.',
  'nice keep 1litre quantity more in stocks',
  'Quality is good but price is more',
  'ok',
  'good brand',
  'Sunflower oil is good for health, earlier I was mostly using soyabean oil which lead to pain in finger joints. Once I switched to sunflower oil the pain went away.',
  'the oil is non sticky in nature very smooth and lighter',
  'oil smells badly after single use',
  'Its a good product but hea

In [14]:
len(reviews)

633